#### Imports

In [98]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
# Import the mean_absolute_error function
from sklearn.metrics import mean_absolute_error

# Calculate the MSE

import pandas as pd

# print working directory
import os

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [5]:

print(os.getcwd())
# Load the CSV file
file_path = '../Data/Metro_sales_count_now_uc_sfrcondo_month.csv'
df = pd.read_csv(file_path)

# Print the first 5 rows
print(df.head())


/Users/bermo/github/capstone/real_estate_analysis/Notebooks
   RegionID  SizeRank       RegionName RegionType StateName  2008-02-29  \
0    102001         0    United States    country       NaN    200263.0   
1    394913         1     New York, NY        msa        NY      8416.0   
2    753899         2  Los Angeles, CA        msa        CA      4141.0   
3    394463         3      Chicago, IL        msa        IL      5570.0   
4    394514         4       Dallas, TX        msa        TX      4875.0   

   2008-03-31  2008-04-30  2008-05-31  2008-06-30  ...  2023-11-30  \
0    231896.0    257463.0    283461.0    296547.0  ...    258258.0   
1      8883.0      9809.0     10081.0     10685.0  ...     11143.0   
2      5044.0      6065.0      6868.0      7222.0  ...      5119.0   
3      6908.0      7230.0      7931.0      8781.0  ...      7022.0   
4      5519.0      5988.0      6679.0      6672.0  ...      4625.0   

   2023-12-31  2024-01-31  2024-02-29  2024-03-31  2024-04-30  2024-

In [40]:
# read in realtor zip code data
file_path = '../Data/realtor_zip_monthly_historical.csv'
df_realtor = pd.read_csv(file_path)

# print the first 5 rows
print(df_realtor.head())

# print the shape
print(df_realtor.shape)


/var/folders/sp/_mjrzq3j7cx5cnmtdrl_njsr0000gn/T/ipykernel_97905/3801920293.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_realtor = pd.read_csv(file_path)


  month_date_yyyymm  postal_code            zip_name  median_listing_price  \
0            202408      32343.0          midway, fl              304950.0   
1            202408      62896.0  west frankfort, il              125000.0   
2            202408      56477.0          sebeka, mn              302400.0   
3            202408      25315.0      charleston, wv               65000.0   
4            202408      76454.0          gorman, tx              184500.0   

   median_listing_price_mm  median_listing_price_yy  active_listing_count  \
0                   0.0168                  -0.1028                  14.0   
1                   0.3999                   0.6955                  29.0   
2                  -0.1186                  -0.1919                  13.0   
3                  -0.1613                   0.1017                   5.0   
4                  -0.0027                   0.1535                   3.0   

   active_listing_count_mm  active_listing_count_yy  median_days_on_

In [11]:
# list out all the columns
print(df_realtor.columns)

# list out all the columns
print(df_realtor.columns)

# list out all the columns
print(df_realtor.columns)



Index(['month_date_yyyymm', 'postal_code', 'zip_name', 'median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'active_listing_count', 'active_listing_count_mm',
       'active_listing_count_yy', 'median_days_on_market',
       'median_days_on_market_mm', 'median_days_on_market_yy',
       'new_listing_count', 'new_listing_count_mm', 'new_listing_count_yy',
       'price_increased_count', 'price_increased_count_mm',
       'price_increased_count_yy', 'price_reduced_count',
       'price_reduced_count_mm', 'price_reduced_count_yy',
       'pending_listing_count', 'pending_listing_count_mm',
       'pending_listing_count_yy', 'median_listing_price_per_square_foot',
       'median_listing_price_per_square_foot_mm',
       'median_listing_price_per_square_foot_yy', 'median_square_feet',
       'median_square_feet_mm', 'median_square_feet_yy',
       'average_listing_price', 'average_listing_price_mm',
       'average_listing_price_yy', 'total_listing_cou

In [41]:
# get count of unique values of quality_flag
print(df_realtor['quality_flag'].value_counts())


quality_flag
1.0    1384727
0.0    1117903
Name: count, dtype: int64


## MODELING

#### Drop columns

In [61]:
# List of columns to drop based on assumptions about redundancy or lack of utility for prediction
columns_to_drop = [
    'postal_code', 'zip_name', 'quality_flag'
]

# Drop the columns from the DataFrame
df_cleaned = df_realtor.drop(columns=columns_to_drop)

In [90]:
    # Drop rows with NaN values in 'month_date_yyyymm'
df_cleaned = df_cleaned.dropna(subset=['month_date_yyyymm'])

# drop rows with NaN values in the median_days_on_market
df_cleaned = df_cleaned.dropna(subset=['median_days_on_market'])

# remove rows where month_date_yyyymm is "quality_flag = 1:  year-over-year figures may be impacted
df_cleaned = df_cleaned[df_cleaned['month_date_yyyymm'] != 'quality_flag = 1:  year-over-year figures may be impacted']
# print the counts of all the unique values in month_date_yyyymm
print(df_cleaned['month_date_yyyymm'].value_counts())



month_date_yyyymm
202108    42663
202009    41025
202008    39786
201910    39576
201810    38115
          ...  
202103    26564
202203    26553
202102    26522
202202    26342
201607     2550
Name: count, Length: 99, dtype: int64


#### Separate Categorical and Numerical Columns

postal_code and zip_name are categorical features, but since we are predicting aggregated data at the ZIP code level, we drop these.All other columns are numerical, including percentage change columns.

In [91]:
# Define numerical columns (drop the target column 'median_days_on_market')
numerical_columns = [col for col in df_cleaned.columns if col not in ['median_days_on_market']]

#### Define Preprocessing Pipelines

In [92]:
# Pipeline for numerical features: handles missing values and scales data
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with mean
    ('scaler', StandardScaler())  # Standardize numerical features
])

# Combine pipelines for numerical columns
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_columns)
])

#### Create Train and Test Set

In [93]:
# Define features (X) and target (y)
X = df_cleaned.drop(columns=['median_days_on_market'])  # Features matrix
y = df_cleaned['median_days_on_market']  # Target variable

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print(X_test.head())



        month_date_yyyymm  median_listing_price  median_listing_price_mm  \
356814             202308              185000.0                   0.0000   
1572240            202002              169700.0                   0.0000   
2838298            201607              287000.0                      NaN   
1186400            202102              168450.0                   0.0060   
1498231            202004               91225.0                   0.0732   

         median_listing_price_yy  active_listing_count  \
356814                   -0.0508                   1.0   
1572240                   2.7711                   1.0   
2838298                      NaN                  18.0   
1186400                   0.1139                   1.0   
1498231                   0.7376                   4.0   

         active_listing_count_mm  active_listing_count_yy  \
356814                      0.00                    -0.50   
1572240                     0.00                     0.00   
2838298    

#### Preprocessing

In [94]:
# Fit the preprocessing pipeline to the training data
X_train_preprocessed = preprocessor.fit_transform(X_train)

# Transform the test set using the fitted pipeline
X_test_preprocessed = preprocessor.transform(X_test)

# head of the preprocessed data
print(pd.DataFrame(X_train_preprocessed).head())

         0         1             2             3         4             5   \
0 -1.676364 -0.190553 -3.020467e-18  1.106673e-18 -0.526032 -1.022253e-17   
1  1.249785  0.190059 -6.012560e-03 -6.008202e-03 -0.057721 -3.727604e-01   
2  0.807068 -0.006724 -6.101943e-03 -6.086513e-03 -0.387273 -6.232082e-01   
3 -1.292676 -0.072421 -3.020467e-18  1.106673e-18  0.046348 -1.022253e-17   
4  1.237136 -0.367720 -6.407557e-03 -6.122056e-03 -0.421963  1.753983e+00   

             6             7             8         9   ...            26  \
0  1.702948e-17 -1.136905e-16  2.484970e-17 -0.592997  ... -7.725606e-18   
1 -2.784798e-02 -1.234424e-01  8.490754e-02 -0.338043  ... -3.630853e-03   
2 -6.590699e-01 -1.946566e-01  2.536978e-02 -0.423027  ... -1.431927e-02   
3  1.702948e-17 -1.136905e-16  2.484970e-17 -0.423027  ... -7.725606e-18   
4  1.841153e+00 -1.253735e-01  2.394148e-01 -0.423027  ... -1.022761e-02   

         27            28            29        30            31            32  \

### Modeling

In [95]:
# how many NaNs are in the training labels
print(y_train.isna().sum())

# percentage of NaNs in the training labels
print(y_train.isna().sum() / len(y_train) * 100)

# percentage of NaNs in the test labels
print(y_test.isna().sum() / len(y_test) * 100)

# Initialize the Linear Regression model

0
0.0
0.0


In [100]:
# Initialize the Linear Regression model
linear_model = LinearRegression()

# Fit the model using the preprocessed training data
linear_model.fit(X_train_preprocessed, y_train)

# Predict on the test set
y_pred = linear_model.predict(X_test_preprocessed)

# Calculate the MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate the MSE
mse = mean_squared_error(y_test, y_pred)

# print the MAE and MSE
print(f"MAE: {mae}")
print(f"MSE: {mse}")

# print the R2 score
r2 = r2_score(y_test, y_pred)
print(f"R2: {r2}")



MAE: 32.36845423487267
MSE: 2225.85716059081
R2: 0.22261650727141724
